# Stock Performance Tracker

This code allows a user to input any stocks that they would like to view performance over a specific period of time. It can be used to simplify viewing and comparing various stocks over the same time window.

There are 3 functions included in this notebook.

1. `get_data()` which uses the `yfinance` python package to pull stock data from yahoo finance for a specific perfiod of time.
2. `plotting_stocks()` takes in a dataframe of all stock information, and creates interactive subplots using the `plotly` package.
3. `portfolio_stocks()` takes in user inputs for any number of stock ticker symbols to be plotted and time period using the 2 user-defined functions above to display final graphs.


**Disclaimer:** *Stock prices are not sourced from all markets and may be delayed depending on real time data available from [yahoo finance](https://finance.yahoo.com/). Information is provided 'as is' and solely for informational purposes, not for trading purposes or advice.*


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import yfinance as yf
import os
import contextlib
from plotly.subplots import make_subplots
import plotly.graph_objects as go

## Function to Retrieve Stock Prices

In [ ]:
def get_data(ls, period = '5y'):
    """
    Pulls stock information for a specifc period length

    Keyword arguments:
    ls -- list of stock ticker symbols
    period -- length of stock data: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
    """
        
    global date_range
    
    date_range = period
    
    stock_list = []

    for ticker in ls:
        df = pd.DataFrame(yf.download(ticker, period = period,auto_adjust = True))
        df['Ticker'] = '{}'.format(ticker)
        stock_list.append(df)

    dataframe = pd.concat(stock_list, axis=0).reset_index()
    
    return dataframe

## Function to Plot Stock Prices

In [ ]:
def plotting_stocks(dataframe):
    """
    Plotting each stock ticker into a subplots.

    Keyword arguments:
    dataframe -- stock dataframe to graph each ticker
    """
    
    # setting variables
    stock_cnt = dataframe['Ticker'].nunique()
    stocks = sorted(list(dataframe['Ticker'].value_counts().index))

    #subplot formating
    fig = make_subplots(
                        rows=stock_cnt, cols=1,
                        shared_xaxes=True,
                        vertical_spacing = 0.01,
                        horizontal_spacing=0.085,
                        x_title='Days',
                        y_title='Closing Stock Price ($)')

    # plotting each subplot
    for index, stock in enumerate(stocks):
        fig.add_trace(
                      go.Scatter(x= dataframe[dataframe['Ticker'] == stock]['Date'], 
                                 y = dataframe[dataframe['Ticker'] == stock]['Close'], 
                                 name = stock
                                  ),
                      row=index+1, col=1
        )

        fig.add_annotation(x = dataframe[dataframe['Ticker'] == stock]['Date'].max(),# - timedelta(days=10), 
                           y = dataframe[dataframe['Ticker'] == stock]['Close'].min(),
                           text = stock,
                           showarrow = False,
                           font=dict(
                                    family="Arial",
                                    size=30,
                                    color="black"
                                    ),
                           row = index+1, col = 1)

    # formating main plot
    fig.update_layout(height = 300*stock_cnt, width = 1000, 
                      showlegend = False,
                      title = dict(
                                   text = '{} - Daily Closing Prices'.format(date_range),
                                   font = dict(
                                               family = 'Arial',
                                               size = 30,
                                               color = 'black')
                                     ),
                        font = dict(
                                    family = 'Arial',
                                    size = 16,
                                    color = 'black')
                      )

    fig.show()

## Function to Input Stock Tickers

In [ ]:
def portfolio_stocks():
    
    """
    Full function to take in stock ticker inputs from users and output graphs
    
    """
        
    # saving output globally
    global stock_df

    print('''
        You can track each stock by entering the company's ticker symbol.
        -----------------------------------------------------------------
''')
    
    ls = []

    while True:

#         print()
        stock = input('''  
        Enter ticker symbol or press enter to end.
        ------------------------------------------
        ''').upper()

        if stock == '':
            break
        else:
            ls.append(stock)

    while True:
        time = str(input('''
        Enter time period you would like to view.
        Valid inputs: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        ----------------------------------------------------
        '''))
        
        valid = ['1d', '5d', '1mo', '3mo', '6mo',
                 '1y', '2y', '5y', '10y', 'ytd', 'max']

        if time not in valid:
            True
        else:
            stock_df = get_data(ls, period=time)
            break

    order = ['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume']

    stock_df = stock_df.loc[:, order]

    return plotting_stocks(stock_df)

## Run Program

In [ ]:
# function to run user inputs
portfolio_stocks()